### 네이버 영화 웹크롤링

In [2]:
from bs4 import BeautifulSoup 
import requests 
import urllib.request
from selenium.webdriver.common.keys import Keys 
from urllib.request import urlretrieve
import time 
import re 
import pandas as pd
import numpy as np
from selenium import webdriver
from multiprocessing import Pool
import multiprocessing
import concurrent.futures

In [3]:
def getPageLinks(): 
    
    links = [] 
    path='C:/Users/qkrdb/Downloads/chromedriver.exe'
    chrome = webdriver.Chrome(path)
    
    for page in range(0, 19):
        if (page==2 or page==8):
            continue
        chrome.get('https://movie.naver.com/movie/sdb/rank/rmovie.nhn?sel=cnt&date=20200907&tg=' + str(page + 1))
        chrome.implicitly_wait(3)
        
        for i in range(0, 10):
            pathmovie = '//*[@id="old_content"]/table/tbody/tr[' + str(i + 2) + ']/td[2]/div/a'
            link =chrome.find_element_by_xpath(pathmovie).get_attribute('href')
            links.append(link)
            if page==7:
                break
            elif page==9:
                if (i>=6):
                    break
    chrome.close()
    return links

In [4]:
naver_links = getPageLinks()

In [7]:
print(len(naver_links))

111


In [8]:
naver_links

['https://movie.naver.com/movie/bi/mi/basic.nhn?code=189623',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=150198',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=194644',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=189000',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=192608',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=175092',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=142308',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=163834',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=62310',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=185917',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=182234',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=142785',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=184311',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=195668',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=183116',
 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=62586',
 'https://

In [6]:
#중복제거
naver_links = list(set(naver_links))

In [9]:
def getinfo(links):
    n=1
    path='C:/Users/qkrdb/Downloads/chromedriver.exe'
    chrome = webdriver.Chrome(path)
    movie_title = '//*[@id="content"]/div[1]/div[2]/div[1]/h3/a[1]' 
    movie_director = '//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[2]/p/a'
    movie_country = '//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[1]/p/span[2]/a'
    movie_ganre = '//*[@id="content"]/div[1]/div[2]/div[1]/dl/dd[1]/p/span[1]/a' 
    movie_info = 'div.video div.story_area p.con_tx'
    movie_review = 'div.article div.review div.user_tx_area'

    global movie_list
    movie_list=pd.DataFrame()

    for link in links:
        chrome.get(link)
        # 암묵적으로 웹 자원 로드를 위해 3초까지 기다려 준다. implicitly.wait()
        chrome.implicitly_wait(3)
        DB = pd.DataFrame()
        
        title = chrome.find_element_by_xpath(movie_title).get_attribute('text')
        director =chrome.find_element_by_xpath(movie_director).get_attribute('text')
        country = chrome.find_element_by_xpath(movie_country).get_attribute('text')
        ganre1 = chrome.find_elements_by_xpath(movie_ganre)[0].text
        try:
            ganre2 = chrome.find_elements_by_xpath(movie_ganre)[1].text
            ganre = ganre1 + ", " + ganre2
        except:
            ganre = ganre1
            pass
        try:
            info = chrome.find_element_by_css_selector(movie_info).text
            info = info.replace('\n', ' ')
        except:
            info = 'no exist'
            pass
         
        DB['title']=[title]
        DB['director']=[director]
        DB['country']=[country]
        DB['ganre']=[ganre]
        DB['info']=[info]
        num = str(n)
        DB['number']=[num]
        movie_list = movie_list.append(DB,ignore_index=True)
        n+=1

In [10]:
getinfo(naver_links)

In [11]:
movie_list

,title,director,country,ganre,info,number
0,남매의 여름밤,윤단비,한국,드라마,"방학 동안, 아빠와 함께 할아버지 집에서 지내게 된 남매 옥주와 동주, 그렇게 오...",1
1,너의 이름은.,신카이 마코토,일본,"애니메이션, 드라마","아직 만난 적 없는 너를, 찾고 있어 천년 만에 다가오는 혜성 기적이 시작된...",2
2,기기괴괴 성형수,조경훈,한국,"공포, 스릴러",지금부터 기적의 성형수 사용 방법을 알려드리겠습니다. 본인의 얼굴을 거울로 확인해...,3
3,나이브스 아웃,라이언 존슨,미국,"미스터리, 스릴러",베스트셀러 미스터리 작가가 85세 생일에 숨진 채 발견된다. 그의 죽음의 원인을 ...,4
4,범죄도시 2,이상용,한국,"범죄, 액션",no exist,5
5,먼 훗날 우리,유약영,중국,"드라마, 멜로/로맨스","2007년 춘절, 귀향하는 기차에서 처음 만나 친구가 된 ‘린젠칭’(정백연)과 ‘팡...",6
6,하이에나 로드,폴 그로스,캐나다,"드라마, 전쟁","목표물을 인간으로 보지 않는 냉혹하지만, 최고 실력을 갖춘 저격수. 군인이지만 직접...",7
7,원더 우먼 1984,패티 젠킨스,미국,"액션, 모험",아마존 데미스키라 왕국의 공주이자 신이 만든 가장 완벽한 히어로인 원더우먼의 활약을...,8
8,하우스,오바야시 노부히코,일본,"공포, 판타지",no exist,9
9,반도,연상호,한국,"액션, 드라마","[전대미문의 재난 그 후 4년 폐허의 땅으로 다시 들어간다!] 4년 전, 나...",10


In [13]:
movie_list.to_csv("naverinfo.csv", encoding='utf-8-sig')

In [14]:
def getreview(links):
    n=1
    path='C:/Users/qkrdb/Downloads/chromedriver.exe'
    chrome = webdriver.Chrome(path)
    
    movie_review = 'div.article div.review div.user_tx_area'
    global movie
    global DB
    movie=pd.DataFrame()
    
    
    for link in links:
        if link=='https://movie.naver.com/movie/bi/mi/basic.nhn?code=184516':
            DB['number'] = [num]
            n+=1
            DB['review'] = 'no exist'
            movie = movie.append(DB,ignore_index=True)
            continue #블랙팬서 남주인공이 고인이라 이미지가 삭제돼서 없습니다.
        DB = pd.DataFrame()
        link = link.replace("basic", "review")
        print(link)
        reviews=''
        chrome.get(link)
        chrome.implicitly_wait(3)
        
        for j in range(1, 11):
            try:
                path1 = '//*[@id="reviewTab"]/div/div/ul/li[' + str(j) + ']/a'
                chrome.find_element_by_xpath(path1).click()
            except:
                continue
            chrome.implicitly_wait(3)
            r = chrome.find_elements_by_css_selector(movie_review) 
            
            for i in range(len(r)):
                reviews += r[i].text
            chrome.find_element_by_xpath('//*[@id="content"]/div[1]/div[4]/div[1]/div[1]/a').click()
            chrome.implicitly_wait(3)
        reviews.replace('\n',' ')
        num = str(n)
        DB['number'] = [num]
        n+=1
        DB['review'] = [reviews]
        movie = movie.append(DB,ignore_index=True)
    chrome.close()

In [15]:
getreview(naver_links)

https://movie.naver.com/movie/bi/mi/review.nhn?code=189623
https://movie.naver.com/movie/bi/mi/review.nhn?code=150198
https://movie.naver.com/movie/bi/mi/review.nhn?code=194644
https://movie.naver.com/movie/bi/mi/review.nhn?code=189000
https://movie.naver.com/movie/bi/mi/review.nhn?code=192608
https://movie.naver.com/movie/bi/mi/review.nhn?code=175092
https://movie.naver.com/movie/bi/mi/review.nhn?code=142308
https://movie.naver.com/movie/bi/mi/review.nhn?code=163834
https://movie.naver.com/movie/bi/mi/review.nhn?code=62310
https://movie.naver.com/movie/bi/mi/review.nhn?code=185917
https://movie.naver.com/movie/bi/mi/review.nhn?code=182234
https://movie.naver.com/movie/bi/mi/review.nhn?code=142785
https://movie.naver.com/movie/bi/mi/review.nhn?code=184311
https://movie.naver.com/movie/bi/mi/review.nhn?code=195668
https://movie.naver.com/movie/bi/mi/review.nhn?code=183116
https://movie.naver.com/movie/bi/mi/review.nhn?code=62586
https://movie.naver.com/movie/bi/mi/review.nhn?code=193799

In [16]:
movie

,number,review
0,1,반지하라도 ‘우리’집이 있었다. 이제는 ‘우리’집이 아닌 할아버지 댁으로 들어가...
1,2,부산국제영화제에서 처음 봤던 영화는 '너의 이름은' 이라는 애니였다. 친구가 표를 ...
2,3,영화 기기괴괴 성형수를\n시사회로 보고왔어요\n\n 영화 기기괴괴 성형수는 물과 성...
3,4,"나이브스 아웃 범인은 우리안에 있다 추리 스릴러\n\n나이브스 아웃, 줄거리 결말 ..."
4,5,
5,6,"먼 훗날 우리\n감독\n유약영\n출연\n정백연, 주동우, 톈좡좡\n개봉\n2018 ..."
6,7,정말 잘 만든 영화....\n감동이 넘칩니다...\n추천.....정말 좋은 명화로 ...
7,8,‘원더우먼 2’ 촬영 종료...\n갤 가돗 “2020년에 봐요”\n[할리웃톡]\n ...
8,9,"집이 주인공인 헌팅,헌티드힐과 명맥을 같이 하는 영화로, 오바야시 감독..."
9,10,사람이 살고 있었구나 여기!\n- 영화 반도에서..\n ☆ 프롤로그\n\n여름에...


In [68]:
navercombine = pd.merge(movie_list, movie, on = "number")

In [69]:
navercombine

,title,director,country,ganre,info,number,review
0,남매의 여름밤,윤단비,한국,드라마,"방학 동안, 아빠와 함께 할아버지 집에서 지내게 된 남매 옥주와 동주, 그렇게 오...",1,반지하라도 ‘우리’집이 있었다. 이제는 ‘우리’집이 아닌 할아버지 댁으로 들어가...
1,너의 이름은.,신카이 마코토,일본,"애니메이션, 드라마","아직 만난 적 없는 너를, 찾고 있어 천년 만에 다가오는 혜성 기적이 시작된...",2,부산국제영화제에서 처음 봤던 영화는 '너의 이름은' 이라는 애니였다. 친구가 표를 ...
2,기기괴괴 성형수,조경훈,한국,"공포, 스릴러",지금부터 기적의 성형수 사용 방법을 알려드리겠습니다. 본인의 얼굴을 거울로 확인해...,3,영화 기기괴괴 성형수를\n시사회로 보고왔어요\n\n 영화 기기괴괴 성형수는 물과 성...
3,나이브스 아웃,라이언 존슨,미국,"미스터리, 스릴러",베스트셀러 미스터리 작가가 85세 생일에 숨진 채 발견된다. 그의 죽음의 원인을 ...,4,"나이브스 아웃 범인은 우리안에 있다 추리 스릴러\n\n나이브스 아웃, 줄거리 결말 ..."
4,범죄도시 2,이상용,한국,"범죄, 액션",no exist,5,
5,먼 훗날 우리,유약영,중국,"드라마, 멜로/로맨스","2007년 춘절, 귀향하는 기차에서 처음 만나 친구가 된 ‘린젠칭’(정백연)과 ‘팡...",6,"먼 훗날 우리\n감독\n유약영\n출연\n정백연, 주동우, 톈좡좡\n개봉\n2018 ..."
6,하이에나 로드,폴 그로스,캐나다,"드라마, 전쟁","목표물을 인간으로 보지 않는 냉혹하지만, 최고 실력을 갖춘 저격수. 군인이지만 직접...",7,정말 잘 만든 영화....\n감동이 넘칩니다...\n추천.....정말 좋은 명화로 ...
7,원더 우먼 1984,패티 젠킨스,미국,"액션, 모험",아마존 데미스키라 왕국의 공주이자 신이 만든 가장 완벽한 히어로인 원더우먼의 활약을...,8,‘원더우먼 2’ 촬영 종료...\n갤 가돗 “2020년에 봐요”\n[할리웃톡]\n ...
8,하우스,오바야시 노부히코,일본,"공포, 판타지",no exist,9,"집이 주인공인 헌팅,헌티드힐과 명맥을 같이 하는 영화로, 오바야시 감독..."
9,반도,연상호,한국,"액션, 드라마","[전대미문의 재난 그 후 4년 폐허의 땅으로 다시 들어간다!] 4년 전, 나...",10,사람이 살고 있었구나 여기!\n- 영화 반도에서..\n ☆ 프롤로그\n\n여름에...


In [41]:
#navercombine1.loc[navercombine1['review']=='no exist', 'review']
#navercombine1.loc[navercombine1['review']=='', 'review']

### 전처리

In [70]:
# 리뷰없는 경우 줄거리로 대체
navercombine.loc[4, 'review'] = navercombine.loc[4, 'info']
navercombine.loc[29, 'review'] = navercombine.loc[29, 'info']
navercombine.loc[50, 'review'] = navercombine.loc[50, 'info']
navercombine.loc[53, 'review'] = navercombine.loc[53, 'info']
navercombine.loc[65, 'review'] = navercombine.loc[65, 'info']
navercombine.loc[86, 'review'] = navercombine.loc[86, 'info']

In [71]:
navercombine.loc[navercombine['review']=='no exist', 'review']

4    no exist
Name: review, dtype: object

In [72]:
navercombine = navercombine.drop(navercombine.index[4])

In [73]:
navercombine

,title,director,country,ganre,info,number,review
0,남매의 여름밤,윤단비,한국,드라마,"방학 동안, 아빠와 함께 할아버지 집에서 지내게 된 남매 옥주와 동주, 그렇게 오...",1,반지하라도 ‘우리’집이 있었다. 이제는 ‘우리’집이 아닌 할아버지 댁으로 들어가...
1,너의 이름은.,신카이 마코토,일본,"애니메이션, 드라마","아직 만난 적 없는 너를, 찾고 있어 천년 만에 다가오는 혜성 기적이 시작된...",2,부산국제영화제에서 처음 봤던 영화는 '너의 이름은' 이라는 애니였다. 친구가 표를 ...
2,기기괴괴 성형수,조경훈,한국,"공포, 스릴러",지금부터 기적의 성형수 사용 방법을 알려드리겠습니다. 본인의 얼굴을 거울로 확인해...,3,영화 기기괴괴 성형수를\n시사회로 보고왔어요\n\n 영화 기기괴괴 성형수는 물과 성...
3,나이브스 아웃,라이언 존슨,미국,"미스터리, 스릴러",베스트셀러 미스터리 작가가 85세 생일에 숨진 채 발견된다. 그의 죽음의 원인을 ...,4,"나이브스 아웃 범인은 우리안에 있다 추리 스릴러\n\n나이브스 아웃, 줄거리 결말 ..."
5,먼 훗날 우리,유약영,중국,"드라마, 멜로/로맨스","2007년 춘절, 귀향하는 기차에서 처음 만나 친구가 된 ‘린젠칭’(정백연)과 ‘팡...",6,"먼 훗날 우리\n감독\n유약영\n출연\n정백연, 주동우, 톈좡좡\n개봉\n2018 ..."
6,하이에나 로드,폴 그로스,캐나다,"드라마, 전쟁","목표물을 인간으로 보지 않는 냉혹하지만, 최고 실력을 갖춘 저격수. 군인이지만 직접...",7,정말 잘 만든 영화....\n감동이 넘칩니다...\n추천.....정말 좋은 명화로 ...
7,원더 우먼 1984,패티 젠킨스,미국,"액션, 모험",아마존 데미스키라 왕국의 공주이자 신이 만든 가장 완벽한 히어로인 원더우먼의 활약을...,8,‘원더우먼 2’ 촬영 종료...\n갤 가돗 “2020년에 봐요”\n[할리웃톡]\n ...
8,하우스,오바야시 노부히코,일본,"공포, 판타지",no exist,9,"집이 주인공인 헌팅,헌티드힐과 명맥을 같이 하는 영화로, 오바야시 감독..."
9,반도,연상호,한국,"액션, 드라마","[전대미문의 재난 그 후 4년 폐허의 땅으로 다시 들어간다!] 4년 전, 나...",10,사람이 살고 있었구나 여기!\n- 영화 반도에서..\n ☆ 프롤로그\n\n여름에...
10,오케이 마담,이철하,한국,"코미디, 액션",극강의 쫄깃함으로 빠른 완판을 기록하는 꽈배기 맛집 사장 '미영'은 컴퓨터 수리 ...,11,다들 태풍&코로나 조심하세요. 태풍 마이삭이 지나간지 얼마나 됐다고 태풍 하이선도 ...


In [74]:
navercombine.to_csv("navercombine.csv", encoding='utf-8-sig')

In [75]:
movie.to_csv("naverreview.csv", encoding='utf-8-sig')

In [76]:
def getimg(links):
    n=1
    try:
        path='C:/Users/qkrdb/Downloads/chromedriver.exe'
        div = webdriver.Chrome(path)
        for link in links:
            if link=='https://movie.naver.com/movie/bi/mi/basic.nhn?code=184516':
                n+=1
                continue #블랙팬서 남주인공이 고인이라 이미지가 삭제돼서 없습니다.
            div.get(link)
            pathimg = "C:/Users/qkrdb/네이버영화이미지/"
            div.implicitly_wait(3)
            src = div.find_element_by_xpath('//*[@id="content"]/div[1]/div[2]/div[2]/a/img').get_attribute('src')
            print(src)
            print(pathimg + str(n) + ".jpg")
            urlretrieve(src, pathimg + str(n) + ".jpg")
            n+=1

    except:
         print(isbn + ' 저장 실패')
    
    div.close()

In [78]:
getimg(naver_links)

https://movie-phinf.pstatic.net/20200721_100/1595307510654a6eNC_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/1.jpg
https://movie-phinf.pstatic.net/20171222_70/1513920001726sikUX_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/2.jpg
https://movie-phinf.pstatic.net/20200901_11/1598922866258hETJp_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/3.jpg
https://movie-phinf.pstatic.net/20191119_163/1574125979767APJb7_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/4.jpg
https://movie-phinf.pstatic.net/20200901_262/1598942851401DGhNQ_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/5.jpg
https://movie-phinf.pstatic.net/20180516_300/15264574387234QNai_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/6.jpg
https://movie-phinf.pstatic.net/20160812_199/1470987772569t1JoL_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/7.jpg
https://movie-phinf.pstatic.net/20190613_281/1560405907356YRR3v_JPEG/movie_image.jpg?

https://movie-phinf.pstatic.net/20200821_165/1597998176466dc4sY_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/64.jpg
https://movie-phinf.pstatic.net/20200116_23/1579154974413LvtIf_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/65.jpg
https://movie-phinf.pstatic.net/20180710_211/1531198746645nsdKN_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/67.jpg
https://movie-phinf.pstatic.net/20190821_138/1566362458427V6Usp_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/68.jpg
https://movie-phinf.pstatic.net/20200901_160/1598948570961a9BWq_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/69.jpg
https://movie-phinf.pstatic.net/20200902_264/1599023779693070wc_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/70.jpg
https://movie-phinf.pstatic.net/20190417_250/1555465284425i6WQE_JPEG/movie_image.jpg?type=m203_290_2
C:/Users/qkrdb/네이버영화이미지/71.jpg
https://movie-phinf.pstatic.net/20171218_47/1513561221134wvRja_JPEG/movie_ima